# Import all modules needed

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# From Mitre

## Get the techniques associated with each ransomware

In [ ]:
enterprise_df = pd.read_html("https://attack.mitre.org/techniques/enterprise/")

In [ ]:
data_encrypt_site = "https://attack.mitre.org/techniques/T1486/"

soup = BeautifulSoup(urlopen(data_encrypt_site).read())
sites = []

for row in soup.findAll('table')[0].tbody.findAll('tr'):
  first_column = row.findAll('td')[0].contents
  if first_column[1]['href'][1] == "s":
    sites.append("https://attack.mitre.org"+first_column[1]['href'])
print(str(len(sites)) + " ransomware")

36 ransomware


In [ ]:
df = []
ransomware_names = []
for site in sites:
  temp = pd.read_html(site)
  for t in temp:
    try:
      if t.iloc[0].ID[0]=="T":
        t.name=t.Use[0].split(" ")[0]
        t["Ransomware"] = t.Use[0].split(" ")[0]
        df.append(t)
        ransomware_names.append((t.Use[0].split(" ")[0]))
    except AttributeError:
      continue
print(len(ransomware_names))
print(ransomware_names)

36
['Avaddon', 'Babuk', 'Bad', 'BitPaymer', 'Clop', 'Conti', 'Cuba', 'DEATHRANSOM', 'Diavol', 'Egregor', 'EKANS', 'FIVEHANDS', 'HELLOKITTY', 'JCry', 'KillDisk', 'LockerGoga', 'Maze', 'MegaCortex', 'Netwalker', 'NotPetya', 'Pay2Key', 'ProLock', 'Pysa', 'Ragnar', 'REvil', 'RobbinHood', 'Ryuk', 'SamSam', 'Seth-Locker', 'Shamoon', 'SynAck', 'ThiefQuest', 'WannaCry', 'WastedLocker', 'Xbash', 'XCSSET']


In [ ]:
techniques = enterprise_df[0].dropna().set_index("ID")
techniques = techniques.drop("ID.1",axis=1)

In [ ]:
enterprise_tactics_site = "https://attack.mitre.org/tactics/enterprise/"
soup = BeautifulSoup(urlopen(enterprise_tactics_site).read())
tactics_sites = []
for row in soup.findAll('table')[0].tbody.findAll('tr'):
  first_column = row.findAll('td')[0].contents
  tactics_sites.append([row.findAll('td')[1].contents[1].text,row.findAll('td')[0].contents[1].text,"https://attack.mitre.org"+first_column[1]['href']])
for i in range(len(tactics_sites)):
  temp = pd.read_html(tactics_sites[i][2])
  tactics_sites[i].append(temp[0]["ID"].dropna().to_list())
tactics_sites[0]

['Reconnaissance',
 'TA0043',
 'https://attack.mitre.org/tactics/TA0043',
 ['T1595',
  'T1592',
  'T1589',
  'T1590',
  'T1591',
  'T1598',
  'T1597',
  'T1596',
  'T1593',
  'T1594']]

In [ ]:
# finally attach a tactic to each technique
for i in range(len(tactics_sites)):
  techniques.at[tactics_sites[i][3],"Tactic"] = tactics_sites[i][0]
techniques

,Name,Description,Tactic
ID,,,
T1548,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Defense Evasion
T1134,Access Token Manipulation,Adversaries may modify access tokens to operat...,Defense Evasion
T1531,Account Access Removal,Adversaries may interrupt availability of syst...,Impact
T1087,Account Discovery,Adversaries may attempt to get a listing of ac...,Discovery
T1098,Account Manipulation,Adversaries may manipulate accounts to maintai...,Persistence
...,...,...,...
T1497,Virtualization/Sandbox Evasion,Adversaries may employ various means to detect...,Discovery
T1600,Weaken Encryption,Adversaries may compromise a network device’s ...,Defense Evasion
T1102,Web Service,"Adversaries may use an existing, legitimate ex...",Command and Control


In [ ]:
ransomware_techniques_merge = pd.DataFrame(dtype=str)
for site in df:
  ransomware_techniques_merge = ransomware_techniques_merge.append(site)
ransomware_techniques_merge = pd.merge(ransomware_techniques_merge,techniques,on="ID")
ransomware_techniques_merge.rename(columns = {"Name_x":"Specific Technique","Name_y":"Technique"},inplace=True)

In [ ]:
ransomware_techniques_merge_two = ransomware_techniques_merge
ransomware_techniques_merge_two["index_true"] = ransomware_techniques_merge_two.index.map(str) + "i"
temp = ransomware_techniques_merge_two.pop("index_true")
ransomware_techniques_merge_two.insert(0,"index_true",temp)
ransomware_techniques_merge_two.head()

,index_true,Domain,ID,ID.1,Specific Technique,Use,Ransomware,Technique,Description,Tactic
0,0i,Enterprise,T1548,.002,Abuse Elevation Control Mechanism: Bypass User...,Avaddon bypasses UAC using the CMSTPLUA COM in...,Avaddon,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Defense Evasion
1,1i,Enterprise,T1548,.002,Abuse Elevation Control Mechanism: Bypass User...,Bad Rabbit has attempted to bypass UAC and gai...,Bad,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Defense Evasion
2,2i,Enterprise,T1548,.002,Abuse Elevation Control Mechanism: Bypass User...,BitPaymer can suppress UAC prompts by setting ...,BitPaymer,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Defense Evasion
3,3i,Enterprise,T1548,.002,Abuse Elevation Control Mechanism: Bypass User...,Shamoon attempts to disable UAC remote restric...,Shamoon,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Defense Evasion
4,4i,Enterprise,T1548,.002,Abuse Elevation Control Mechanism: Bypass User...,WastedLocker can perform a UAC bypass if it is...,WastedLocker,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Defense Evasion


## The techniques that are used the most in general

In [ ]:
most_used_techniques = pd.DataFrame(ransomware_techniques_merge.ID.value_counts()).rename_axis('ID').join(techniques).rename(columns={"ID":"Counts"})
most_used_techniques.head(15)

,Counts,Name,Description,Tactic
ID,,,,
T1486,36,Data Encrypted for Impact,Adversaries may encrypt data on target systems...,Impact
T1059,22,Command and Scripting Interpreter,Adversaries may abuse command and script inter...,Execution
T1490,22,Inhibit System Recovery,Adversaries may delete or remove built-in oper...,Impact
T1027,20,Obfuscated Files or Information,Adversaries may attempt to make an executable ...,Defense Evasion
T1489,18,Service Stop,Adversaries may stop or disable services on a ...,Impact
T1106,18,Native API,Adversaries may interact with the native OS ap...,Execution
T1083,16,File and Directory Discovery,Adversaries may enumerate files and directorie...,Discovery
T1562,16,Impair Defenses,Adversaries may maliciously modify components ...,Defense Evasion
T1082,15,System Information Discovery,An adversary may attempt to get detailed infor...,Discovery


### Tactics that are used the most

In [ ]:
tactics_ransomware = dict(tuple(most_used_techniques.groupby('Tactic')))
tacticts_and_counts = {}
for tactic in tactics_ransomware:
  tacticts_and_counts[tactic] = tactics_ransomware[tactic].Counts.sum()

tacticts_and_counts
for tactic, count in sorted(tacticts_and_counts.items(), key=lambda x: x[1],reverse=True):
  print(str(count)+"\t\t"+str(tactic))

140		Defense Evasion
125		Discovery
93		Impact
64		Execution
30		Command and Control
22		Privilege Escalation
15		Lateral Movement
8		Credential Access
7		Collection
4		Initial Access
3		Exfiltration
3		Persistence
1		Resource Development


## Techniques for each malware

In [ ]:
ransomware_grouped = {}
for ransomware, ransomware_techniques in ransomware_techniques_merge.groupby('Ransomware'):
  ransomware_grouped[ransomware] = ransomware_techniques

In [ ]:
ransomware_name = "Avaddon"
ransomware_grouped[ransomware_name].sort_values(by="Tactic")

,index_true,Domain,ID,ID.1,Specific Technique,Use,Ransomware,Technique,Description,Tactic
0,0i,Enterprise,T1548,.002,Abuse Elevation Control Mechanism: Bypass User...,Avaddon bypasses UAC using the CMSTPLUA COM in...,Avaddon,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Defense Evasion
174,174i,Enterprise,T1027,T1027,Obfuscated Files or Information,Avaddon has used encrypted strings.[2],Avaddon,Obfuscated Files or Information,Adversaries may attempt to make an executable ...,Defense Evasion
96,96i,Enterprise,T1562,.001,Impair Defenses: Disable or Modify Tools,Avaddon looks for and attempts to stop anti-ma...,Avaddon,Impair Defenses,Adversaries may maliciously modify components ...,Defense Evasion
69,69i,Enterprise,T1140,T1140,Deobfuscate/Decode Files or Information,Avaddon has decrypted encrypted strings.[2],Avaddon,Deobfuscate/Decode Files or Information,Adversaries may use Obfuscated Files or Inform...,Defense Evasion
134,134i,Enterprise,T1112,T1112,Modify Registry,Avaddon modifies several registry keys for per...,Avaddon,Modify Registry,Adversaries may interact with the Windows Regi...,Defense Evasion
80,80i,Enterprise,T1083,T1083,File and Directory Discovery,Avaddon has searched for specific files prior ...,Avaddon,File and Directory Discovery,Adversaries may enumerate files and directorie...,Discovery
236,236i,Enterprise,T1016,T1016,System Network Configuration Discovery,Avaddon can collect the external IP address of...,Avaddon,System Network Configuration Discovery,Adversaries may look for details about the net...,Discovery
162,162i,Enterprise,T1135,T1135,Network Share Discovery,Avaddon has enumerated shared folders and mapp...,Avaddon,Network Share Discovery,Adversaries may look for folders and drives sh...,Discovery
194,194i,Enterprise,T1057,T1057,Process Discovery,Avaddon has collected information about runnin...,Avaddon,Process Discovery,Adversaries may attempt to get information abo...,Discovery
226,226i,Enterprise,T1614,.001,System Location Discovery: System Language Dis...,Avaddon checks for specific keyboard layouts a...,Avaddon,System Location Discovery,Adversaries may gather information in an attem...,Discovery


Sort the tactics according to the matrix representation

In [ ]:
tactic_hierarchy = [item[0] for item in tactics_sites]
print("tactic_hierarchy")
print(tactic_hierarchy)
print()

sorted_tactics = {}
for ransomware in ransomware_grouped:
  sorted_tactics[ransomware] = []
  for tactic in tactic_hierarchy:
    if tactic in ransomware_grouped[ransomware]["Tactic"].unique():
        sorted_tactics[ransomware].append(tactic)

tactic_hierarchy
['Reconnaissance', 'Resource Development', 'Initial Access', 'Execution', 'Persistence', 'Privilege Escalation', 'Defense Evasion', 'Credential Access', 'Discovery', 'Lateral Movement', 'Collection', 'Command and Control', 'Exfiltration', 'Impact']



The next code shows the tactics that are most prevalent in all the ransomware.

- Impact is present in all as it includs Data Encryption
- Execution

In [ ]:
tactics_used_count_normalised = {}

for ransomware in sorted_tactics:
  for tactic in sorted_tactics[ransomware]:
    try: 
      tactics_used_count_normalised[tactic] += 1
    except KeyError:
      tactics_used_count_normalised[tactic] = 1

tactics_used_count_normalised
sorted(tactics_used_count_normalised.items(), key=lambda item: item[1],reverse=True)

[('Impact', 36),
 ('Execution', 34),
 ('Defense Evasion', 32),
 ('Discovery', 30),
 ('Privilege Escalation', 16),
 ('Command and Control', 15),
 ('Lateral Movement', 10),
 ('Credential Access', 5),
 ('Collection', 4),
 ('Initial Access', 3),
 ('Exfiltration', 3),
 ('Persistence', 2),
 ('Resource Development', 1)]

In [ ]:
print("Techniques for each tactic\n")
for tactic in sorted(tactics_used_count_normalised.items(), key=lambda item: item[1],reverse=True)[:7]:
  print(tactic[0])
  print("---------------------------")
  print(ransomware_techniques_merge[ransomware_techniques_merge["Tactic"] == tactic[0]]["Technique"].value_counts().rename(tactic[0]).head())
  print("---------------------------\n")

# print(ransomware_techniques_merge[ransomware_techniques_merge["Tactic"] == "Impact"]["Technique"].value_counts().rename("Technique"))
# print(ransomware_techniques_merge[ransomware_techniques_merge["Tactic"] == "Execution"]["Technique"].value_counts())

Techniques for each tactic

Impact
---------------------------
Data Encrypted for Impact    36
Inhibit System Recovery      22
Service Stop                 18
Data Destruction              5
System Shutdown/Reboot        5
Name: Impact, dtype: int64
---------------------------

Execution
---------------------------
Command and Scripting Interpreter     22
Native API                            18
Windows Management Instrumentation    11
System Services                        8
User Execution                         3
Name: Execution, dtype: int64
---------------------------

Defense Evasion
---------------------------
Obfuscated Files or Information            20
Impair Defenses                            16
Indicator Removal on Host                  14
Masquerading                               13
Deobfuscate/Decode Files or Information    11
Name: Defense Evasion, dtype: int64
---------------------------

Discovery
---------------------------
File and Directory Discovery              

## Pickle Mitre database

In [ ]:
import pickle
with open('mitre_ransomware.pickle', 'wb') as handle:
    pickle.dump(ransomware_techniques_merge, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('mitre_ransomware.pickle', 'rb') as handle:
    b = pickle.load(handle)

ransomware_techniques_merge_pickle = b

print(ransomware_techniques_merge == ransomware_techniques_merge_pickle)

     index_true  Domain    ID  ID.1  Specific Technique   Use  Ransomware  \
0          True    True  True  True                True  True        True   
1          True    True  True  True                True  True        True   
2          True    True  True  True                True  True        True   
3          True    True  True  True                True  True        True   
4          True    True  True  True                True  True        True   
..          ...     ...   ...   ...                 ...   ...         ...   
510        True    True  True  True                True  True        True   
511        True    True  True  True                True  True        True   
512        True    True  True  True                True  True        True   
513        True    True  True  True                True  True        True   
514        True    True  True  True                True  True        True   

     Technique  Description  Tactic  
0         True         True    True  

# from John's Database and Malware Bazaar

## Get 10 instances of a ransomware

With a list of ransomware as input get 10 instances of that from John's Database

In [ ]:
import requests
import json
from ipywidgets import IntProgress, Label, HBox
import threading

In [ ]:
# ransomware list
# ransomware_names is the variable for all the ransomware
ransomware =  ["Conti"]
ransomware_dictionary = {}
number_of_samples = 10
number_of_ransomware = len(ransomware_names)

Try to find 10 hashes from bazaar

In [ ]:
f = IntProgress(min=0, max=number_of_ransomware, description="Scraping: ") # instantiate the bar
g = Label(value=f'0/36')
box = HBox([f,g])
display(box) # display the bar
counter = 0
for ransomware_name in ransomware_names:
  ransomware_dictionary[ransomware_name] = {}
  data = {
    'query': 'get_siginfo',
    'signature': ''+ ransomware_name +'',
  }

  response = requests.post('https://mb-api.abuse.ch/api/v1/', data=data, timeout=3)
  json_response =  json.loads(response.content.decode("utf-8", "ignore"))
  try:
    for search_result in json_response["data"][:10]:
      ransomware_dictionary[ransomware_name][search_result["sha256_hash"]] = {}
  except KeyError:
    pass
  f.value += 1 
  g.value = f'Ransomware: {f.value}/{number_of_ransomware}'
print(ransomware_dictionary)

{'Avaddon': {'d841d0a10e8b6885f1b8e1282c70e88d4f74471fbbe1b4b6f29b4ca238b1e8cb': {}, '1d7051ad6ad4f278e54651e289fb01c034261bdb3e366ccea8c55fa834979118': {}, '14db90c83f43d96505e48dc86efa5c57be8474fc993f00fb7d14d5ba4e21c341': {}, 'f70c8b49a7e79a338bfe4a3db293f6be2001afdc37a45bbaab5ecf0c7dea01a0': {}, '48d7cd572f14aed7a90d6b66097a885a889e6e7416a6aaa2eb442706ff661275': {}, '0ff4058f709d278ed662719b9627618c48e7a656c59f6bfecda9081c7cbd742b': {}, '2946ef53c8fec94dcdf9d3a1afc077ee9a3869eacb0879cb082ee0ce3de6a2e7': {}, '46a8c1e768f632d69d06bfbd93932d102965c9e3f7c37d4a92e30aaeca905675': {}, 'c14dd4a0831ea2548e1ddfd54b9704fe8ad0057924ede041c8c064b66690a028': {}, '7b7c16367746efe7583ae46235b2f062ce44602dda990c9a11a730d619b8d365': {}}, 'Babuk': {'393a7a313548a4edc025fb47c6c8e614ecc2b41db880ecb59f20cf238e9a864c': {}, '00fdeee2f48cc21a5b3da49aacda35513a98d671c8309557b37a378ed57c63bd': {}, '80c58ad933073630a37ad8d78739807b1f4b6f5f65900c59c8424b5df11c632b': {}, '25835a890a218fd26bfd8b23696576402b5eb8a

Get links for joesandbox for each hash.

In [ ]:
# %%timeit -r 1 -n 1
# f = IntProgress(min=0, max=number_of_ransomware*number_of_samples, description="Scraping: ") # instantiate the bar
f = IntProgress(min=0, max=113, description="Scraping: ") # instantiate the bar
# g = Label(value=f'0/{number_of_ransomware*number_of_samples}')
g = Label(value=f'0/113')
box = HBox([f,g])
display(box) # display the bar

threads = []

def get_and_set_joesandbox(joesandbox_boiler, ransomware_dictionary, ransomware_name, ransomware_hash):
    # search the hash on joesansbox
    soup = BeautifulSoup(urlopen(joesandbox_boiler+ransomware_hash).read())
    # if a sandbox 
    try:
      ransomware_dictionary[ransomware_name][ransomware_hash] = {"link":'https://www.joesandbox.com'+soup.find('table').tbody.find('tr').find('td').find('a')["href"]}
      # print(ransomware_dictionary[ransomware_name])
      # print(ransomware_dictionary[ransomware_name][ransomware_hash],end=", ")
    except AttributeError:
      del ransomware_dictionary[ransomware_name][ransomware_hash]
    f.value +=1
    g.value = str(f.value) + f'/113'

joesandbox_boiler = 'https://www.joesandbox.com/search?q='
# go through each ransomware
for ransomware_name in ransomware_dictionary:
  # go through each hash of a ransomware
  ransomware_dictionary_keys = list(ransomware_dictionary[ransomware_name].keys())
  print(ransomware_name,end=" ")
  for ransomware_hash in ransomware_dictionary_keys:
    thread = threading.Thread(target=get_and_set_joesandbox, args=(joesandbox_boiler, ransomware_dictionary, ransomware_name, ransomware_hash))
    thread.start()
    threads.append(thread)
  for thread in threads:
    thread.join()
  threads = []
    # break
  # break

# print(ransomware_dictionary)

Avaddon Babuk Bad BitPaymer Clop Conti Cuba DEATHRANSOM Diavol Egregor EKANS FIVEHANDS HELLOKITTY JCry KillDisk LockerGoga Maze MegaCortex Netwalker NotPetya Pay2Key ProLock Pysa Ragnar REvil RobbinHood Ryuk SamSam Seth-Locker Shamoon SynAck ThiefQuest WannaCry WastedLocker Xbash XCSSET 

In [ ]:
for ransomware_name in ransomware_dictionary:
  print(ransomware_name, ransomware_dictionary[ransomware_name])

Avaddon {'d841d0a10e8b6885f1b8e1282c70e88d4f74471fbbe1b4b6f29b4ca238b1e8cb': {'link': 'https://www.joesandbox.com/analysis/579491/0/html'}, '48d7cd572f14aed7a90d6b66097a885a889e6e7416a6aaa2eb442706ff661275': {'link': 'https://www.joesandbox.com/analysis/415806/0/html'}, '0ff4058f709d278ed662719b9627618c48e7a656c59f6bfecda9081c7cbd742b': {'link': 'https://www.joesandbox.com/analysis/411217/0/html'}, '2946ef53c8fec94dcdf9d3a1afc077ee9a3869eacb0879cb082ee0ce3de6a2e7': {'link': 'https://www.joesandbox.com/analysis/411218/0/html'}, '46a8c1e768f632d69d06bfbd93932d102965c9e3f7c37d4a92e30aaeca905675': {'link': 'https://www.joesandbox.com/analysis/411220/0/html'}, 'c14dd4a0831ea2548e1ddfd54b9704fe8ad0057924ede041c8c064b66690a028': {'link': 'https://www.joesandbox.com/analysis/372587/0/html'}, '7b7c16367746efe7583ae46235b2f062ce44602dda990c9a11a730d619b8d365': {'link': 'https://www.joesandbox.com/analysis/290019/0/html'}}
Babuk {'393a7a313548a4edc025fb47c6c8e614ecc2b41db880ecb59f20cf238e9a864c':

## Get Matrix, Joe Sandbox Signatures, System Behaviour and  Network Behaviour

can get patrix from IOC report

In [ ]:
ransomware_dictionary_non_empty = {}
for ransomware_name in ransomware_dictionary:
  # print(ransomware_name," ",len(ransomware_dictionary[ransomware_name]))
  if len(ransomware_dictionary[ransomware_name]) > 0:
    ransomware_dictionary_non_empty[ransomware_name] = ransomware_dictionary[ransomware_name]
    print(ransomware_name,ransomware_dictionary_non_empty[ransomware_name])
print("ransomware_dictionary_non_empty length: ", len(ransomware_dictionary_non_empty))
ransomware_dictionary = ransomware_dictionary_non_empty
ransomware_dictionary

Avaddon {'d841d0a10e8b6885f1b8e1282c70e88d4f74471fbbe1b4b6f29b4ca238b1e8cb': {'link': 'https://www.joesandbox.com/analysis/579491/0/html'}, '48d7cd572f14aed7a90d6b66097a885a889e6e7416a6aaa2eb442706ff661275': {'link': 'https://www.joesandbox.com/analysis/415806/0/html'}, '0ff4058f709d278ed662719b9627618c48e7a656c59f6bfecda9081c7cbd742b': {'link': 'https://www.joesandbox.com/analysis/411217/0/html'}, '2946ef53c8fec94dcdf9d3a1afc077ee9a3869eacb0879cb082ee0ce3de6a2e7': {'link': 'https://www.joesandbox.com/analysis/411218/0/html'}, '46a8c1e768f632d69d06bfbd93932d102965c9e3f7c37d4a92e30aaeca905675': {'link': 'https://www.joesandbox.com/analysis/411220/0/html'}, 'c14dd4a0831ea2548e1ddfd54b9704fe8ad0057924ede041c8c064b66690a028': {'link': 'https://www.joesandbox.com/analysis/372587/0/html'}, '7b7c16367746efe7583ae46235b2f062ce44602dda990c9a11a730d619b8d365': {'link': 'https://www.joesandbox.com/analysis/290019/0/html'}}
Babuk {'393a7a313548a4edc025fb47c6c8e614ecc2b41db880ecb59f20cf238e9a864c':

{'Avaddon': {'d841d0a10e8b6885f1b8e1282c70e88d4f74471fbbe1b4b6f29b4ca238b1e8cb': {'link': 'https://www.joesandbox.com/analysis/579491/0/html'},
  '48d7cd572f14aed7a90d6b66097a885a889e6e7416a6aaa2eb442706ff661275': {'link': 'https://www.joesandbox.com/analysis/415806/0/html'},
  '0ff4058f709d278ed662719b9627618c48e7a656c59f6bfecda9081c7cbd742b': {'link': 'https://www.joesandbox.com/analysis/411217/0/html'},
  '2946ef53c8fec94dcdf9d3a1afc077ee9a3869eacb0879cb082ee0ce3de6a2e7': {'link': 'https://www.joesandbox.com/analysis/411218/0/html'},
  '46a8c1e768f632d69d06bfbd93932d102965c9e3f7c37d4a92e30aaeca905675': {'link': 'https://www.joesandbox.com/analysis/411220/0/html'},
  'c14dd4a0831ea2548e1ddfd54b9704fe8ad0057924ede041c8c064b66690a028': {'link': 'https://www.joesandbox.com/analysis/372587/0/html'},
  '7b7c16367746efe7583ae46235b2f062ce44602dda990c9a11a730d619b8d365': {'link': 'https://www.joesandbox.com/analysis/290019/0/html'}},
 'Babuk': {'393a7a313548a4edc025fb47c6c8e614ecc2b41db880e

### Get the Matrices

In [ ]:
# f = IntProgress(min=0, max=number_of_ransomware*number_of_samples, description="Scraping: ") # instantiate the bar
f = IntProgress(min=0, max=113, description="Scraping: ") # instantiate the bar
# g = Label(value=f'0/{number_of_ransomware*number_of_samples}')
g = Label(value=f'0/113')
box = HBox([f,g])
display(box) # display the bar

def get_table(link):
  import re
  how = urlopen(link,timeout=5).read()
  how = str(how)
  try:
    # start = how.find('<table class="table table-bordered hd joeTable nohover"')
    start = how.find('<tr class="mitreattack-head"')
    end = how.find('</span></td></tr></tbody></table>',start)
    # print(start,end=" ")
    # print(end,end=" ")
    if start == -1 or end == -1:
      f.value +=1
      g.value = f'{f.value}/113'
      return pd.DataFrame()
    table = '<table class="table table-bordered"><thead>' + how[start:end] + '</span></td></tr></tbody></table>'
    # print(table)
  except (ParserError,OSError):
    f.value +=1
    g.value = f'{f.value}/113'
    return pd.DataFrame()
  table = pd.read_html(table)[0]
  for i in range(table.shape[0]):
    for j in range(table.shape[1]):
      try:
        numbers = list(re.findall(r'\d+',table.iloc[i][j]))
        if len(numbers) == 0:
          table.iloc[i][j] = ""
          continue
        # print(numbers)
        numbers = [int(number) for number in numbers[0]]
        technique = re.findall(r'[^\d]+',table.iloc[i][j])[0]
        # print(technique)
        table.iloc[i][j] = [technique] + [sum(numbers)]
      except (TypeError, ValueError):
        # print("hello")
        table.iloc[i][j] = ""
        pass
  f.value +=1
  g.value = f'{f.value}/113'
  return table

## this one has the hashes as the key of a dictionary
def df_to_dictionary(df):
  to_return = {}
  df_dict = df.to_dict()
  for key in df_dict:
    for key_2 in df_dict[key]:
      if df_dict[key][key_2] != "":
        try:
          to_return[key][df_dict[key][key_2][0]] = df_dict[key][key_2][1]
        except:
          to_return[key] = {}
          to_return[key][df_dict[key][key_2][0]] = df_dict[key][key_2][1]
  return to_return
  
def assign_techniques_to_ransomware_hash(ransomware_name, hash):
  ransomware_dictionary[ransomware_name][hash]["Techniques"] = df_to_dictionary(get_table(ransomware_dictionary[ransomware_name][hash]['link']))

def assign_techniques_to_ransomware():
  threads = []
  ransomware_names = list(ransomware_dictionary.keys())
  for ransomware_name in ransomware_names:
    hashes = ransomware_dictionary[ransomware_name].keys()
    print(ransomware_name, end=" ")
    # break
    for hash in hashes:
      # print(ransomware_dictionary[ransomware_name][hash]['link'], end=" ")
      thread = threading.Thread(target=assign_techniques_to_ransomware_hash, args=(ransomware_name, hash))
      thread.start()
      threads.append(thread)
    for thread in threads:
      thread.join()
    threads = []
    # break
      # print()
    # break

assign_techniques_to_ransomware()

# ransomware_dictionary[keys[0]]

# get_table("https://www.joesandbox.com/analysis/639487/0/html")

Avaddon Babuk Clop Conti Cuba DEATHRANSOM Diavol Egregor EKANS HELLOKITTY Maze Netwalker Pay2Key Ragnar REvil Ryuk WannaCry WastedLocker 

In [ ]:
for i in ransomware_dictionary:
  print(i)

Avaddon
Babuk
Clop
Conti
Cuba
DEATHRANSOM
Diavol
EKANS
Egregor
HELLOKITTY
Maze
Netwalker
Pay2Key
REvil
Ragnar
Ryuk
WannaCry
WastedLocker


### Get the Signatures

In [ ]:
def link_to_execution(ransomware_name, hash):
  return ransomware_dictionary[ransomware_name][hash]['link'][:-4]+"executive"

def set_signature_to_hash(ransomware_name,hash):
  ransomware_dictionary[ransomware_name][hash]["Signatures"] = get_signatures(ransomware_name,hash)
  f.value +=1
  g.value = f'{f.value}/113'

In [ ]:
def get_signatures(ransomware_name, hash):
  link = link_to_execution(ransomware_name,hash)
  # print(link," ")
  import re
  how = urlopen(link,timeout=5).read()
  soup = BeautifulSoup(how)
  try:
    categories_table = soup.find_all("div", class_="signature-expert-wrapper")[0]
    # print("Type:",1)
  except:
    # print("signature",ransomware_name,hash)
    return get_signatures_second(ransomware_name,hash,soup)
  
  dictionary = {}
  for categories_table in categories_table.find_all("table"):
    try:
      dictionary[categories_table.find("h3").text] = []
    except:
      continue
    categories_attribute = categories_table.find_next_sibling()
    try:
      while categories_attribute.name != "table":
        dictionary[categories_table.find("h3").text].append({"name":categories_attribute.find("span").text,"status":categories_attribute['class'][-1]})
        categories_attribute = categories_attribute.find_next_sibling().find_next_sibling()
    except AttributeError:
      pass
  # print(dictionary)
  return dictionary

In [ ]:
def get_signatures_second(ransomware_name,hash,soup):
  try:
    categories_table = soup.find_all("div", {"id":"collapseSigs"})[0]
    # print("Type:",2)
  except:
    # print("signature 2",ransomware_name,hash)
    return get_signatures_third(ransomware_name,hash,soup)
  
  dictionary = {}
  # print("number of categories:",len(categories_table.find_all("table", class_="table table-bordered bd")))
  for categories_table in categories_table.find_all("table", class_="table table-bordered bd"):
    try:
      category_name = categories_table.find("h3").text.strip()[:-1]
      # print(category_name)
      dictionary[category_name] = []
    except:
      continue
    categories_attribute = categories_table.find_next_sibling()
    try:
      while True:
        # dictionary[categories_table.find("h3").text].append({"name":categories_attribute.find("span").text,"status":categories_attribute['class'][-1]})
        # print(categories_attribute.find("b").text,end=" ")
        # print("hello")
        # print(categories_attribute.find("tr"))
        status_colour = categories_attribute.find("tr")["style"].strip()
        # print(status_colour,end=" ")
        if status_colour == "background-color:#c52508; color: #b41a00":
          status = "dangerous"
        elif status_colour == "background-color:#ea2200;":
          status = "malicious"
        elif status_colour == "background-color:#f55c00;":
          status = "suspicious"
        elif status_colour in ["background-color:#3ea63e; display: none","background-color:#C2F0C2; display: none"]:
          status = "clean"
        elif status_colour in ["background-color:#008fff; display: none","background-color:#c6e6ff; display: none"]:
          status = "info"
        else:
          raise UserWarning('Status Not Found')
        # print(status)
        dictionary[category_name].append({"name":categories_attribute.find("b").text ,"status":status})
        categories_attribute = categories_attribute.find_next_sibling().find_next_sibling()
        # raise UserWarning('Exit Early')
    except (AttributeError,KeyError):
      pass
  return dictionary

In [ ]:
def get_signatures_third(ransomware_name,hash,soup):
  categories_table = soup.find_all("div", {"id":"signatures"})[0]
  dictionary = {}

  for categories_table in categories_table.find_all("table", class_="table table-bordered bd")+categories_table.find_all("table", class_="expertOnly table table-bordered bd"):
    # print("Hllooooooo")
    try:
      category_name = categories_table.find("h3").text.strip()[:-1]
      # print("Type:",3)
      dictionary[category_name] = []
    except:
      # print("signature 3",ransomware_name,hash)
      return get_signatures_four(ransomware_name,hash)
    # print(category_name)
    categories_attribute = categories_table.find_next_sibling()
    try:
      while True:
        # dictionary[categories_table.find("h3").text].append({"name":categories_attribute.find("span").text,"status":categories_attribute['class'][-1]})
        # print(categories_attribute.find("b").text,end=" ")
        # print("hello")
        # print(categories_attribute.find("tr"))
        # print("\t",categories_attribute.find("b").text,end=" ")
        status_colour = categories_attribute.find("tr")["style"].strip()
        # print(status_colour,end=" ")
        if status_colour in ["background-color:#c52508; color: #b41a00","background-color:#c52508; color: #f9f3f3"]:
          status = "dangerous"
        elif status_colour == "background-color:#f59180;":
          status = "malicious"
        elif status_colour == "background-color:#ff9b5f; display: none":
          status = "suspicious"
        elif status_colour in ["background-color:#3ea63e; display: none","background-color:#C2F0C2; display: none","background-color: rgb(194, 240, 194);"]:
          status = "clean"
        elif status_colour in ["background-color:#008fff; display: none","background-color:#c6e6ff; display: none","background-color: rgb(198, 230, 255);"]:
          status = "info"
        else:
          raise UserError('Status Not Found')
          # return get_signatures_four(ransomware_name,hash)
        # print(status)
        dictionary[category_name].append({"name":categories_attribute.find("b").text ,"status":status})
        categories_attribute = categories_attribute.find_next_sibling().find_next_sibling()
        # raise UserWarning('Exit Early')
    except (AttributeError,KeyError):
      pass

  # raise UserError("Done")

  # print(dictionary)
  return dictionary

In [ ]:
def get_signatures_four(ransomware_name,hash):
  # print("Type:",4)
  # print("jadkflasjdf",ransomware_name)
  # print("sdfg",hash)
  # print(ransomware_dictionary[ransomware_name][hash]["link"])
  how = urlopen(ransomware_dictionary[ransomware_name][hash]["link"],timeout=5).read()
  soup = BeautifulSoup(how)

  categories_table = soup
  dictionary = {}

  for categories_table in categories_table.find_all("table", class_="table table-bordered bd")+categories_table.find_all("table", class_="expertOnly table table-bordered bd"):
    # print("Hllooooooo")
    try:
      category_name = categories_table.find("h4").text.strip()[:-1]
      dictionary[category_name] = []
    except:
      continue
    # print(category_name)
    categories_attribute = categories_table.find_next_sibling()
    try:
      while True:
        # dictionary[categories_table.find("h3").text].append({"name":categories_attribute.find("span").text,"status":categories_attribute['class'][-1]})
        # print(categories_attribute.find("b").text,end=" ")
        # print("hello")
        # print(categories_attribute.find("tr"))
        print(categories_attribute.find("b").text[0:0],end="") ##### this print statement needs to be here to poke the categories_attribute so it doesn't sleep on the job
        # categories_attribute.find("b")
        status_colour = categories_attribute.find("tr")["style"].strip()
        # print(status_colour,end=" ")
        if status_colour == "background-color:#c52508; color: #f9f3f3":
          status = "dangerous"
        elif status_colour == "background-color:#f59180;":
          status = "malicious"
        elif status_colour == "background-color:#ff9b5f;":
          status = "suspicious"
        elif status_colour in ["background-color:#C2F0C2;","background-color:#C2F0C2; display: none"]:
          status = "clean"
        elif status_colour in ["background-color:#c6e6ff;","background-color:#c6e6ff; display: none"]:
          status = "info"
        else:
          raise UserWarning('Status Not Found')
        # print(status)
        dictionary[category_name].append({"name":categories_attribute.find("b").text ,"status":status})
        categories_attribute = categories_attribute.find_next_sibling().find_next_sibling()
        # raise UserWarning('Exit Early')
    except (AttributeError,KeyError):
      pass

  # raise UserError("Done")

  # print(dictionary)
  return dictionary

In [ ]:
from numpy import True_
f = IntProgress(min=0, max=113, description="Scraping: ") # instantiate the bar
g = Label(value=f'0/113')
box = HBox([f,g])
display(box) # display the bar

# print(get_signatures(ransomware_name,hash))
for ransomware_name in ransomware_dictionary:
  print(ransomware_name,end=" ")
  for hash in ransomware_dictionary[ransomware_name]:
    thread = threading.Thread(target=set_signature_to_hash, args=(ransomware_name, hash))
    thread.start()
    threads.append(thread)
  for thread in threads:
    thread.join()
  threads = []

Avaddon Babuk Clop Conti Cuba DEATHRANSOM Diavol Egregor EKANS HELLOKITTY Maze Netwalker Pay2Key Ragnar REvil Ryuk WannaCry WastedLocker 

In [ ]:
ransomware_dictionary["REvil"]

{'ea1872b2835128e3cb49a0bc27e4727ca33c4e6eba1e80422db19b505f965bc4': {'link': 'https://www.joesandbox.com/analysis/440765/0/html',
  'Techniques': {'Execution': {'Command and Scripting Interpreter': 1},
   'Defense Evasion': {'File and Directory Permissions Modification': 1},
   'Credential Access': {'OS Credential Dumping': 1},
   'Discovery': {'Security Software Discovery': 2,
    'System Information Discovery': 2},
   'Command and Control': {'Proxy': 1},
   'Network Effects': {'Exploit SS': 7}},
  'Signatures': {'Bitcoin Miner': [{'name': 'Reads CPU information from /sys indicative of miner or evasive malware',
     'status': 'suspicious'}],
   'Networking': [{'name': 'Found Tor onion address', 'status': 'malicious'},
    {'name': 'URLs found in memory or binary data', 'status': 'info'}],
   'Spam, unwanted Advertisements and Ransom Demands': [{'name': 'Creates a notice file (html or txt) to demand a ransom',
     'status': 'malicious'}],
   'System Summary': [{'name': 'Sample has s

## Aggregate all the Techniques for each ransomware type

A dataframe of all the techniques that are used for each ransomware
- __index__: Technique
- ID
- Count

In [ ]:
techniques_and_count_old = pd.DataFrame(columns=["Count"])
for ransomware_name in ransomware_dictionary:
  for hash in ransomware_dictionary[ransomware_name]:
    for tactic in ransomware_dictionary[ransomware_name][hash]["Techniques"]:
      technique = list(ransomware_dictionary[ransomware_name][hash]["Techniques"][tactic].keys())[0]
      cost = ransomware_dictionary[ransomware_name][hash]["Techniques"][tactic][technique]
      try:
        techniques_and_count_old.loc[technique] = techniques_and_count_old.loc[technique]["Count"] + count
      except KeyError:
        techniques_and_count_old.loc[technique] = count
techniques_and_count_old.sort_values(by="Count",ascending=False)

,Count
Exploit SS,108
Encrypted Channel,89
Input Capture,72
Exfiltration Over C,64
Masquerading,59
Process Injection,59
Windows Service,51
System Time Discovery,50
Archive Collected Data,49
Data Encrypted for Impact,35


In [ ]:
techniques_and_count = {}
count = 0
for ransomware_name in ransomware_dictionary:
  for hash in ransomware_dictionary[ransomware_name]:
    for tactic in ransomware_dictionary[ransomware_name][hash]["Techniques"]:
      print(tactic)
      break
    break
  break


Initial Access


In [ ]:
def get_technique_joe(ransomware_dictionary):
  test_dict = {}
  techniques_and_tactics = {}

  for ransomware_name in ransomware_dictionary:
    test_dict[ransomware_name] = {}
    for hash in ransomware_dictionary[ransomware_name]:
      for tactic in ransomware_dictionary[ransomware_name][hash]["Techniques"]:
        # test_dict[tactic] = {}
        technique = list(ransomware_dictionary[ransomware_name][hash]["Techniques"][tactic].keys())[0]
        count = ransomware_dictionary[ransomware_name][hash]["Techniques"][tactic][technique]
        test_dict[ransomware_name][technique] = count
        # print(tactic)
        # print("\t",technique)
        techniques_and_tactics[technique]= tactic
      # raise

  techniques_and_tactics = pd.DataFrame.from_dict(techniques_and_tactics,orient='index',columns=["Tactic"])
  # print(techniques_and_tactics)
  result = techniques_and_tactics.join(pd.DataFrame.from_dict(test_dict,dtype=int).fillna(0).astype(int))
  # print(result.drop("Tactic",axis=1).T.sum())
  result["Sum of occassions on each Ransomware"] = result.drop("Tactic",axis=1).T.sum()
  # print(result.drop("Tactic",axis=1).T.sum())
  # print(result.columns)
  # raise
  result["Number on # Ransomware"] = (result.drop(["Tactic","Sum of occassions on each Ransomware"],axis=1) > 0).T.sum()
  return result
  

techniques_joe = get_technique_joe(ransomware_dictionary)

# techniques_joe.loc["Data from Local System"]
# ransomware_dictionary
techniques_joe[techniques_joe["Sum of occassions on each Ransomware"] == techniques_joe["Number on # Ransomware"]].drop("Tactic",axis=1)
# (techniques_joe.drop("Tactic",axis=1) > 0).loc["Data from Local System"]
# pd.DataFrame(list(techniques_joe["Number on # Ransomware"])).set_index(0)
# techniques_joe

,Avaddon,Babuk,Clop,Conti,Cuba,DEATHRANSOM,Diavol,Egregor,EKANS,HELLOKITTY,Maze,Netwalker,Pay2Key,Ragnar,REvil,Ryuk,WannaCry,WastedLocker,Sum of occassions on each Ransomware,Number on # Ransomware
Replication Through Removable Media,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4
DLL Side-Loading,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,5,5
Taint Shared Content,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,3,3
Deobfuscate/Decode Files or Information,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,2
Access Token Manipulation,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,7,7
Scheduled Task/Job,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,2
Native API,0,1,0,1,0,1,1,1,0,1,1,1,0,0,0,0,0,0,8,8
Query Registry,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,4,4
Registry Run Keys / Startup Folder,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,0,0,7,7
Process Discovery,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2


In [ ]:

find_technique = "Encrypted Channel"

for ransomware in ransomware_dictionary:
  for hash in ransomware_dictionary[ransomware]:
    for tactic in ransomware_dictionary[ransomware][hash]["Techniques"]:
      for technique in ransomware_dictionary[ransomware][hash]["Techniques"][tactic]:
        if technique == find_technique:
          print(ransomware)
          print(tactic)
          print(ransomware_dictionary[ransomware][hash]["link"])

Avaddon
Command and Control
https://www.joesandbox.com/analysis/579491/0/html
Avaddon
Command and Control
https://www.joesandbox.com/analysis/415806/0/html
Avaddon
Command and Control
https://www.joesandbox.com/analysis/411217/0/html
Avaddon
Command and Control
https://www.joesandbox.com/analysis/411218/0/html
Avaddon
Command and Control
https://www.joesandbox.com/analysis/411220/0/html
Avaddon
Command and Control
https://www.joesandbox.com/analysis/372587/0/html
Avaddon
Command and Control
https://www.joesandbox.com/analysis/290019/0/html
Babuk
Command and Control
https://www.joesandbox.com/analysis/666429/0/html
Babuk
Command and Control
https://www.joesandbox.com/analysis/611636/0/html
Babuk
Command and Control
https://www.joesandbox.com/analysis/611634/0/html
Babuk
Command and Control
https://www.joesandbox.com/analysis/611635/0/html
Babuk
Command and Control
https://www.joesandbox.com/analysis/606420/0/html
Babuk
Command and Control
https://www.joesandbox.com/analysis/545011/0/htm

In [ ]:
signature_to_find = "Data Encrypted"
links = {}
for ransomware in ransomware_dictionary:
  for hash in ransomware_dictionary[ransomware]:
    for category in ransomware_dictionary[ransomware][hash]['Signatures']:
      for signature in ransomware_dictionary[ransomware][hash]['Signatures'][category]:
        if signature_to_find in signature['name']:
          links[ransomware] = ransomware_dictionary[ransomware][hash]['link']

print(signature_to_find)
links

Data Encrypted


{}

In [ ]:
ransomware_dictionary["Avaddon"]

{'d841d0a10e8b6885f1b8e1282c70e88d4f74471fbbe1b4b6f29b4ca238b1e8cb': {'link': 'https://www.joesandbox.com/analysis/579491/0/html',
  'Techniques': {'Initial Access': {'Replication Through Removable Media': 1},
   'Execution': {'Windows Management Instrumentation': 2,
    'Native API': 2,
    'Scheduled Task/Job': 1},
   'Persistence': {'DLL Side-Loading': 1, 'Scheduled Task/Job': 1},
   'Privilege Escalation': {'DLL Side-Loading': 1,
    'Access Token Manipulation': 1,
    'Process Injection': 2,
    'Scheduled Task/Job': 1},
   'Defense Evasion': {'Disable or Modify Tools': 1,
    'Deobfuscate/Decode Files or Information': 1,
    'Obfuscated Files or Information': 3,
    'Software Packing': 2,
    'DLL Side-Loading': 1,
    'File Deletion': 1,
    'Masquerading': 1,
    'Virtualization/Sandbox Evasion': 1,
    'Access Token Manipulation': 1,
    'Process Injection': 2},
   'Discovery': {'System Time Discovery': 2,
    'Peripheral Device Discovery': 2,
    'File and Directory Discovery

## Aggregate Signatures

In [ ]:
signatures = {}

signature_ransomware_name_dictionary = {ransomware_name:False for ransomware_name in list(ransomware_dictionary.keys())}
signature_ransomware_name_dictionary["status"] = ""
for ransomware_name in ransomware_dictionary:
  for hash in ransomware_dictionary[ransomware_name]:
    for category in ransomware_dictionary[ransomware_name][hash]["Signatures"]:
      for signature in ransomware_dictionary[ransomware_name][hash]["Signatures"][category]:
        if signature["name"] not in signatures:
          signatures[signature["name"]] = signature_ransomware_name_dictionary.copy()
        signatures[signature["name"]][ransomware_name] = True
        signatures[signature["name"]]["status"] = signature["status"]


signatures_dataframe = pd.DataFrame.from_dict(signatures).T
signatures_sorted_index = signatures_dataframe.T.drop("status").sum().sort_values(ascending=False).index
signatures_dataframe["# of Ransomware"] = signatures_dataframe.T.drop("status").sum()

In [ ]:
signatures_dataframe.loc[signatures_sorted_index][signatures_dataframe["status"] != "info"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Avaddon,Babuk,Clop,Conti,Cuba,DEATHRANSOM,Diavol,Egregor,EKANS,HELLOKITTY,Maze,Netwalker,Pay2Key,Ragnar,REvil,Ryuk,WannaCry,WastedLocker,status,# of Ransomware
Detected potential crypto function,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,suspicious,17
Multi AV Scanner detection for submitted file,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,dangerous,16
"Uses code obfuscation techniques (call, push, ret)",True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,suspicious,16
Uses 32bit PE files,True,True,True,True,True,False,True,False,True,True,True,False,True,True,False,True,True,True,suspicious,14
Binary contains paths to debug symbols,False,True,True,True,True,True,True,True,False,False,True,True,True,True,False,True,True,True,clean,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yara detected Cuba ransomware,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,dangerous,1
Sample contains symbols with suspicious names,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,suspicious,1
PE file contains a COM descriptor data directory,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,clean,1
.NET source code contains potential unpacker,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,malicious,1


In [ ]:
signatures_dataframe

,Avaddon,Babuk,Clop,Conti,Cuba,DEATHRANSOM,Diavol,Egregor,EKANS,HELLOKITTY,Maze,Netwalker,Pay2Key,Ragnar,REvil,Ryuk,WannaCry,WastedLocker,status,# of Ransomware
Multi AV Scanner detection for submitted file,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,dangerous,16
Antivirus / Scanner detection for submitted sample,True,True,True,True,True,False,False,True,False,True,True,True,True,True,False,True,True,True,dangerous,14
Antivirus detection for dropped file,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,dangerous,4
Multi AV Scanner detection for dropped file,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,dangerous,4
Machine Learning detection for sample,True,True,True,True,True,True,True,False,False,True,True,True,False,True,False,True,True,True,malicious,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Accesses ntoskrnl, likely to find offsets for exploits",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,malicious,1
Tries to download HTTP data from a sinkholed server,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,dangerous,1
Yara detected WastedLocker,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,dangerous,1
Creates files in alternative data streams (ADS),False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,malicious,1


## Pickle Joe's dictionary

In [ ]:
import pickle
with open('joedatabase_ransomware.pickle', 'wb') as handle:
    pickle.dump(ransomware_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('joedatabase_ransomware.pickle', 'rb') as handle:
    b = pickle.load(handle)

ransomware_dictionary_pickle = b

print(ransomware_dictionary == ransomware_dictionary_pickle)

True
